In [0]:
%run ../00.set_variables

In [0]:
from pathlib import Path
import shutil
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

current_notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
current_notebook_dir = Path(current_notebook_path).parent

str_dir = str(current_notebook_dir)
print(f"cleaning out directory {str_dir}/genie_space")
#crude but whatevs
try:
   w.workspace.delete(path=f"{str_dir}/genie_space", recursive=True)
   print(f"Deleted directory")
except:
   print("directory does not exist, recreating anyways")

w.workspace.mkdirs(path=f"{str_dir}/genie_space")
print(f"Created directory {str_dir}/genie_space")

In [0]:
# Import necessary modules from the Databricks SDK

from databricks.sdk.service import sql


print("Attempting to find the first available SQL warehouse ID...")

try:
    # 1. List all SQL warehouses
    # The list() method returns an iterator of sql.Warehouse objects.
    # We convert it to a list to enable sorting.
    all_warehouses = list(w.warehouses.list())

    if not all_warehouses:
        print("No SQL warehouses found in this workspace.")
        print("Please ensure you have at least one SQL warehouse created.")
    else:
        # 2. Separate running and stopped warehouses
        running_warehouses = []
        stopped_warehouses = []
        for wh in all_warehouses:
            if wh.state == sql.State.RUNNING:
                running_warehouses.append(wh)
            else:
                stopped_warehouses.append(wh)

        # 3. Sort both lists alphabetically by name
        running_warehouses.sort(key=lambda wh: wh.name.lower())
        stopped_warehouses.sort(key=lambda wh: wh.name.lower())

        first_available_warehouse = None

        if running_warehouses:
            # Prioritize running warehouses
            first_available_warehouse = running_warehouses[0]
            print("Found a running SQL warehouse.")
        elif stopped_warehouses:
            # If no running, take the first stopped one
            first_available_warehouse = stopped_warehouses[0]
            print("No running SQL warehouses found. Selecting the first available (stopped) warehouse.")
        else:
            # This case should ideally be caught by `if not all_warehouses:` but as a fallback
            print("No running or stopped SQL warehouses found.")

        if first_available_warehouse:
            warehouse_id = first_available_warehouse.id
            warehouse_name = first_available_warehouse.name
            warehouse_state = first_available_warehouse.state.value

            print("\n--- First Available SQL Warehouse Details ---")
            print(f"ID: {warehouse_id}")
        else:
            print("Could not determine an available SQL warehouse ID.")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure your Databricks authentication is correctly configured and you have permissions to list SQL warehouses.")


In [0]:
from databricks.sdk import WorkspaceClient
import json
from pathlib import Path

current_notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()

# Use pathlib to handle path manipulation robustly
current_notebook_dir = Path(current_notebook_path).parent
full_dashboard_path = (current_notebook_dir / 'genie_space').resolve()

#Genie rooms don't allow / anymore
ws = WorkspaceClient()
result = ws.api_client.do(method="GET", path="/api/2.0/workspace/get-status", body={"path": str(full_dashboard_path)})

In [0]:

folder_id = result['object_id']
room_payload = {
    "display_name": "Fe Concentrator Genie Space",
    "description": "deployed from api",
    "warehouse_id": f"{first_available_warehouse.id}",
    "table_identifiers": [f"{catalog_name}.{schema_name}.gold_iron_ore_predictions"],
    "parent_folder": f'folders/{folder_id}',
    "run_as_type": "VIEWER"
}

response = ws.api_client.do(
    method="POST",
    path="/api/2.0/data-rooms",
    body=room_payload
)

In [0]:
genie_space_id = response['id']
host_url = w.config.host
genie_space_url = host_url + '/genie/rooms/' + genie_space_id

content = {
  "genie_url" : genie_space_url
}

with open("object_urls.json", "r") as file:
    data = json.load(file)

data.update(content)

with open("object_urls.json", "w") as file:
    json.dump(data, file)